In [ ]:
import pandas as pd
import io

url='https://drive.google.com/file/d/1IjIEhLc9n8eLKeY-yh_YigKVWbhgGBsN/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

result_11 = df.loc[11::12, ['result', 'side', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5']]
result_10 = df.loc[10::12, ['result', 'side', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5']]

merged_result = pd.concat([result_10, result_11])
merged_result = merged_result.sort_index()
merged_result = merged_result.reset_index(level=None)

In [ ]:
def clean_data(data):
    rows_to_drop = set()

    for i in range(0, len(data), 2):
        # Check both rows in the pair (team1 and team2)
        if data.iloc[i, 1:].isnull().any() or data.iloc[i + 1, 1:].isnull().any():
            rows_to_drop.add(i)
            rows_to_drop.add(i + 1)

    # Drop identified rows
    cleaned_data = data.drop(rows_to_drop).reset_index(drop = True)
    return cleaned_data

merged_result = clean_data(merged_result)

In [ ]:
import random
random.seed(10)
n_matches = int(len(merged_result) / 2)
print(int(0.8 * n_matches))
all_indices = [i for i in range(n_matches)]
train_indices = random.sample(all_indices, int(0.8 * n_matches))
train_indices.sort()
print(train_indices)

df_train_indices = []
for x in train_indices:
    df_train_indices.append(merged_result.iloc[2 * x]['index'] // 12)
print(df_train_indices)

In [ ]:
# Finding player stats on each champ
player_champ_wr = {}
player_career_wr = {}
champ_wr = {}

player1 = df.loc[0::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player2 = df.loc[1::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player3 = df.loc[2::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player4 = df.loc[3::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player5 = df.loc[4::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player6 = df.loc[5::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player7 = df.loc[6::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player8 = df.loc[7::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player9 = df.loc[8::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player10 = df.loc[9::12, ['playerid','champion', 'result', 'kills', 'deaths', 'assists']]
player_results = pd.concat([player1, player2, player3, player4, player5, player6, player7, player8, player9, player10])
player_results = player_results.sort_index()
player_results[:10]

for index, row in player_results.iterrows():
    if index // 12 in df_train_indices:
        player = row['playerid']
        if player not in player_career_wr:
            player_career_wr[player] = [0, 0]
        player_career_wr[player][0] += row['result']
        player_career_wr[player][1] += 1

        if player not in player_champ_wr:
          player_champ_wr[player] = {}

        champ = row['champion']
        if champ not in player_champ_wr[player]:
          player_champ_wr[player][champ] = [0, 0]
        player_champ_wr[player][champ][0] += row['result']
        player_champ_wr[player][champ][1] += 1

        if champ not in champ_wr:
          champ_wr[champ] = [0, 0]
        champ_wr[champ][0] += row['result']
        champ_wr[champ][1] += 1

print(player_champ_wr['oe:player:e1edfc5cea461399a63cb813cf795cc'])
print(champ_wr)
print(player_career_wr['oe:player:e1edfc5cea461399a63cb813cf795cc'])

In [ ]:
print(len(player_career_wr.keys()))
print(champ_wr.keys())

In [ ]:
league_champions = [ # Not all champs included yet
    "Aatrox", "Ahri", "Akali", "Akshan", "Alistar", "Ambessa", "Amumu", "Anivia", "Annie", "Aphelios", "Ashe",
    "Aurelion Sol", "Aurora", "Azir", "Bard", "Bel'Veth", "Blitzcrank", "Brand", "Braum", "Briar", "Caitlyn", "Camille",
    "Cassiopeia", "Cho'Gath", "Corki", "Darius", "Diana", "Dr. Mundo", "Draven", "Ekko", "Elise",
    "Evelynn", "Ezreal", "Fiddlesticks", "Fiora", "Fizz", "Galio", "Gangplank", "Garen", "Gnar",
    "Gragas", "Graves", "Gwen", "Hecarim", "Heimerdinger", "Hwei", "Illaoi", "Irelia", "Ivern", "Janna",
    "Jarvan IV", "Jax", "Jayce", "Jhin", "Jinx", "K'Sante", "Kai'Sa", "Kalista", "Karma", "Karthus",
    "Kassadin", "Katarina", "Kayle", "Kayn", "Kennen", "Kha'Zix", "Kindred", "Kled", "Kog'Maw", "LeBlanc",
    "Lee Sin", "Leona", "Lillia", "Lissandra", "Lucian", "Lulu", "Lux", "Malphite", "Malzahar",
    "Maokai", "Master Yi", "Milio", "Miss Fortune", "Mordekaiser", "Morgana", "Naafiri", "Nami",
    "Nasus", "Nautilus", "Neeko", "Nidalee", "Nilah", "Nocturne", "Nunu & Willump", "Olaf", "Orianna",
    "Ornn", "Pantheon", "Poppy", "Pyke", "Qiyana", "Quinn", "Rakan", "Rammus", "Rek'Sai", "Rell",
    "Renata Glasc", "Renekton", "Rengar", "Riven", "Rumble", "Ryze", "Samira", "Sejuani", "Senna",
    "Seraphine", "Sett", "Shaco", "Shen", "Shyvana", "Singed", "Sion", "Sivir", "Skarner", "Smolder", "Sona",
    "Soraka", "Swain", "Sylas", "Syndra", "Tahm Kench", "Taliyah", "Talon", "Taric", "Teemo",
    "Thresh", "Tristana", "Trundle", "Tryndamere", "Twisted Fate", "Twitch", "Udyr", "Urgot",
    "Varus", "Vayne", "Veigar", "Vel'Koz", "Vex", "Vi", "Viego", "Viktor", "Vladimir", "Volibear",
    "Warwick", "Wukong", "Xayah", "Xerath", "Xin Zhao", "Yasuo", "Yone", "Yorick", "Yuumi",
    "Zac", "Zed", "Zeri", "Ziggs", "Zilean", "Zoe", "Zyra"
]
champ_cols = ['pick1', 'pick2', 'pick3', 'pick4', 'pick5']

xs = []
ys = []

from gensim.models import Word2Vec
modelW2V = Word2Vec.load("champion_embeddings-2.model")
def get_team_embedding(team, model):
    return np.mean([model.wv[champion] for champion in team], axis=0)

for index1, row in merged_result.iterrows():
    if merged_result.iloc[index1]['index'] % 12 == 11:
        x = []
        teams = [[],[]]
        for k in range(2):
            index = merged_result.iloc[index1]['index'] - k
            comp = [df.iloc[index]['pick1'], df.iloc[index]['pick2'], df.iloc[index]['pick3'], df.iloc[index]['pick4'], df.iloc[index]['pick5']]
#            print(comp)
            bans = [df.iloc[index]['ban1'], df.iloc[index]['ban2'], df.iloc[index]['ban3'], df.iloc[index]['ban4'], df.iloc[index]['ban5']]
            teams[k] = comp

            for i in range(5):
                champ = comp[i]
                if index % 12 == 10:
                    for j in range(index - 10, index - 5):

                        if df.iloc[j]['champion'] == champ:
                            player = df.iloc[j]['playerid']
                            if player in player_champ_wr and champ in player_champ_wr[player]:
                                x.append(player_champ_wr[df.iloc[j]['playerid']][champ][0] / player_champ_wr[df.iloc[j]['playerid']][champ][1])
                                x.append(player_champ_wr[df.iloc[j]['playerid']][champ][1])
                            elif index // 12 in df_train_indices:
                                print(df.iloc[index])
                            if player in player_career_wr:
                                x.append(player_career_wr[df.iloc[j]['playerid']][0] / player_career_wr[df.iloc[j]['playerid']][1])
                            elif index // 12 in df_train_indices:
                                print(df.iloc[index])
                            if champ in champ_wr:
                                x.append(champ_wr[champ][0] / champ_wr[champ][1])
                            elif index // 12 in df_train_indices:
                                print(df.iloc[index])
                else:
                    for j in range(index - 6, index - 1):
                        if df.iloc[j]['champion'] == champ:
                            player = df.iloc[j]['playerid']
                            if player in player_champ_wr and champ in player_champ_wr[player]:
                                x.append(player_champ_wr[df.iloc[j]['playerid']][champ][0] / player_champ_wr[df.iloc[j]['playerid']][champ][1])
                                x.append(player_champ_wr[df.iloc[j]['playerid']][champ][1])
                            elif index // 12 in df_train_indices:
                                print(df.iloc[index])
                            if player in player_career_wr:
                                x.append(player_career_wr[df.iloc[j]['playerid']][0] / player_career_wr[df.iloc[j]['playerid']][1])
                            elif index // 12 in df_train_indices:
                                print(df.iloc[index])
                            if champ in champ_wr:
                                x.append(champ_wr[champ][0] / champ_wr[champ][1])
                            elif index // 12 in df_train_indices:
                                print(df.iloc[index])


            for i in range(5):
                if bans[i] not in champ_wr:
                    x.append(0.5)
                else:
                    x.append(champ_wr[bans[i]][0] / champ_wr[bans[i]][1])
        # adding pre-trained embeddings
        team1_embedding = get_team_embedding(teams[0], modelW2V)
        team2_embedding = get_team_embedding(teams[1], modelW2V)
        x = np.concatenate([x, team1_embedding, team2_embedding])

        if len(x) == 66:
          if len(xs) % 1000 == 0: print(len(xs))
          xs.append([merged_result.iloc[index1]['index'] // 12, x])
          ys.append(row['result'])

In [ ]:
import numpy as np

X_train = []
X_test = []
y_train = []
y_test = []

for i in range(len(xs)):
    if xs[i][0] in train_indices:
        X_train.append(xs[i][1])
        y_train.append(ys[i])
    else:
        X_test.append(xs[i][1])
        y_test.append(ys[i])

print(len(X_train))
print(len(X_test))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, log_loss, accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

model = LogisticRegression()

model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

# calculate precision and recall
precisionLR, recallLR, _ = precision_recall_curve(y_test, y_prob)

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')

# auc
auc = roc_auc_score(y_test, y_pred)
print(f'AUC: {auc}')


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

# calculate precision and recall
precisionRF, recallRF, _ = precision_recall_curve(y_test, y_prob)

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')
auc = roc_auc_score(y_test, y_pred)
print(f'AUC: {auc}')

In [ ]:
gb_reg = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_reg.fit(X_train, y_train)

y_prob = gb_reg.predict_proba(X_test)[:, 1]

y_pred = gb_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

precisionGB, recallGB, _ = precision_recall_curve(y_test, y_prob)
# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')
auc = roc_auc_score(y_test, y_pred)
print(f'AUC: {auc}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch > 5:
        return lr * 0.5  # Reduce LR after 5 epochs
    return lr

lr_scheduler = LearningRateScheduler(scheduler)

xs = pd.DataFrame(xs)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

def create_more_complex_model():
    model = Sequential()

    model.add(Input(shape=(xs.shape[1],)))

    # First Hidden Layer
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Second Hidden Layer
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Third Hidden Layer
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

model = create_more_complex_model()


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


# visualize training results
# Plot accuracy
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.figure(figsize=(8, 6))

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs', fontsize='20')
plt.ylabel('Accuracy', fontsize='20')
plt.legend(fontsize='14')
plt.savefig('embacc.png')
plt.show()

# Plot loss
plt.figure(figsize=(8, 6))

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs',fontsize='20')
plt.ylabel('Loss', fontsize='20')
plt.legend(fontsize='14')
plt.savefig('embloss.png')
plt.show()


In [ ]:
from sklearn.metrics import f1_score

# Predict on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")
from sklearn.metrics import precision_recall_curve

# Calculate precision and recall
precisionNN, recallNN, thresholds = precision_recall_curve(y_test, y_pred_prob)

auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC: {auc:.2f}")

In [ ]:
# Plot the precision-recall curve
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.figure(figsize=(8, 6))
plt.plot(recallLR, precisionLR, label='LR', color='#175E54')
plt.plot(recallRF, precisionRF, label='RF', color='#8C1515')
plt.plot(recallGB, precisionGB, label='GB', color='#4298B5')
plt.plot(recallNN, precisionNN, label='NN', color='#E98300')


plt.xlabel('Recall', fontsize='20')
plt.ylabel('Precision', fontsize='20')
plt.legend(fontsize='16')
plt.tight_layout()
plt.savefig('vectoremb.png')
plt.show()